In [2]:
import pandas as pd
import json

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

In [4]:
input_file = './venezuela-2023-12-05.jsonl'

In [5]:
with open(input_file) as f:
    lines = f.read().splitlines()

In [6]:
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [ ]:
df_inter['json_element'].apply(json.loads)

In [8]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

In [ ]:
df_final.sort_values(['quoteCount'])

In [ ]:
df_final.drop_duplicates(subset=['id_str'], keep='last').sort_values(['retweetCount'], ascending=False)[['user.username', 'retweetCount', 'rawContent']]

In [11]:
import networkx as nx

In [12]:
source = df_final.drop_duplicates(subset=['id_str'], keep='last')

In [13]:
recs = source.to_dict(orient="records")

In [14]:
G = nx.DiGraph()
i = 0
for row_dict in recs:
    if i < 150000:
        dst_node = row_dict['user.username']
        for tgt in row_dict['mentionedUsers']:
            src_node = tgt['username']
            G.add_edge(src_node, dst_node)
        i += 1

In [15]:
G.remove_edges_from(nx.selfloop_edges(G))

In [16]:
G_core = nx.k_core(G, 3)

In [ ]:
G_core.nodes()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10))
nx.draw(G_core, with_labels=True, font_size=8, node_size=10, width=0.5, edge_color=(0, 0, 0, 0.1))
plt.savefig('/Users/atemerev/Documents/net-ua.pdf')
plt.show()

In [ ]:
sorted(G_core.degree, key=lambda x: x[1], reverse=True)

In [ ]:
sorted(G.degree, key=lambda x: x[1], reverse=True)